In [1]:
import os
import sys
import h5py
import argparse
import numpy as np
from collections import Counter
import xml.etree.ElementTree as et 

# Keras imports
from keras.models import Model
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
from keras.layers import Input, Dense, Dropout, AlphaDropout, BatchNormalization, Activation

import matplotlib
matplotlib.use('Agg') # Need this to write images from the GSA servers.  Order matters:
import matplotlib.pyplot as plt # First import matplotlib, then use Agg, then import plt

# IPython imports
from IPython.display import Image

# ML4CVD Imports
from ml4cvd.arguments import parse_args
from ml4cvd.tensor_writer_ukbb import write_tensors
from ml4cvd.tensor_generators import test_train_valid_tensor_generators
from ml4cvd.recipes import train_multimodal_multitask, test_multimodal_multitask
from ml4cvd.models import make_multimodal_to_multilabel_model, train_model_from_generators

%matplotlib inline

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


In [2]:
os.listdir('/mnt/disks/ecg-rest-and-bike-with-trend-tensors/2019-08-13/')[0]

'5732608.hd5'

In [22]:
sys.argv = ['train', 
            '--tensors', '/mnt/disks/ecg-rest-and-bike-with-trend-tensors/2019-08-13/', 
            '--input_tensors', 'ecg_bike_recovery', 
            '--output_tensors','ecg-bike-max-pred-hr-no0', 'ecg-bike-max-hr-no0', 'ecg-bike-resting-hr-no0',
            '--batch_size', '64', 
            '--epochs', '20',
            '--training_steps', '5',
            '--inspect_model',
            '--u_connect',
            '--learning_rate', '0.01',
            '--id', 'ecg_bike_recovery_labeler',
            '--output_folder', '/home/ndiamant/ml/notebooks/recipes_output/ecg_sketch'
           ]
args = parse_args()
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(args.tensor_maps_in,  args.tensor_maps_out,  args.tensors, args.batch_size,   args.valid_ratio, args.test_ratio, args.test_modulo, args.balance_csvs)

2019-08-15 14:54:35,179 - logger:25 - INFO - Logging configuration was loaded. Log messages can be found at /home/ndiamant/ml/notebooks/recipes_output/ecg_sketch/ecg_bike_recovery_labeler/log_2019-08-15_14-54_0.log.
2019-08-15 14:54:35,180 - arguments:180 - INFO - Command Line was:

python train --tensors /mnt/disks/ecg-rest-and-bike-with-trend-tensors/2019-08-13/ --input_tensors ecg_bike_recovery --output_tensors ecg-bike-max-pred-hr-no0 ecg-bike-max-hr-no0 ecg-bike-resting-hr-no0 --batch_size 64 --epochs 20 --training_steps 5 --inspect_model --u_connect --learning_rate 0.01 --id ecg_bike_recovery_labeler --output_folder /home/ndiamant/ml/notebooks/recipes_output/ecg_sketch


2019-08-15 14:54:35,181 - arguments:181 - INFO - Total TensorMaps:4414 Arguments are Namespace(activation='relu', aligned_dimension=16, alpha=0.5, app_csv=None, b_slice_force=None, balance_csvs=[], batch_size=64, bigquery_credentials_file='/mnt/ml4cvd/projects/jamesp/bigquery/bigquery-viewer-credentials.json', bi

In [23]:
model = make_multimodal_to_multilabel_model(args.model_file, args.model_layers, args.model_freeze,
                                                args.tensor_maps_in, args.tensor_maps_out, args.activation,
                                                args.dense_layers, args.dropout, args.mlp_concat, args.conv_layers,
                                                args.max_pools, args.res_layers, args.dense_blocks, args.block_size,
                                                args.conv_bn, args.conv_x, args.conv_y, args.conv_z, args.conv_dropout,
                                                args.conv_width, args.u_connect, args.pool_x, args.pool_y, args.pool_z, args.padding,
                                                args.learning_rate)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_full_ecg_bike_recovery (I (None, 30000, 3)     0                                            
__________________________________________________________________________________________________
conv1d_27 (Conv1D)              (None, 30000, 32)    6848        input_full_ecg_bike_recovery[0][0
__________________________________________________________________________________________________
conv1d_28 (Conv1D)              (None, 30000, 32)    72736       conv1d_27[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_7 (MaxPooling1D)  (None, 15000, 32)    0           conv1d_28[0][0]                  
__________________________________________________________________________________________________
conv1d_30 

In [24]:
# model.predict_generator(generate_test, steps=1)

In [25]:
model = train_model_from_generators(model, generate_train, generate_valid, args.training_steps,
                                        args.validation_steps, args.batch_size, args.epochs, args.patience,
                                        args.output_folder, args.id, args.inspect_model, args.inspect_show_labels)

2019-08-15 14:54:40,326 - models:852 - INFO - Saving architecture diagram to:/home/ndiamant/ml/notebooks/recipes_output/ecg_sketch/ecg_bike_recovery_labeler/architecture_graph_ecg_bike_recovery_labeler.png
Epoch 1/1
2019-08-15 14:54:43,117 - tensor_generators:372 - INFO - Got first error: KeyError while attempting to generate tensor:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/tensor_generators.py", line 80, in multimodal_multitask_generator
    in_batch[tm.input_name()][stats['batch_index']] = tm.tensor_from_file(hd5, dependents)
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/TensorMap.py", line 625, in tensor_from_file
    tensor[:, idx] = hd5[self.group][channel]
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "/usr/local/lib/python3.6/dist-packages/h5py/_hl/group.py", line 262, in __getitem__
    oid = h5o.open(self.id, self._e(n

2019-08-15 14:55:55,797 - tensor_generators:114 - INFO - ValueError while attempting to generate tensor:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/tensor_generators.py", line 86, in multimodal_multitask_generator
    out_batch[tm.output_name()][stats['batch_index']] = tm.tensor_from_file(hd5)
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/TensorMap.py", line 564, in tensor_from_file
    raise ValueError(self.name + ' is a continuous value that cannot be set to 0, but no value was found.')
ValueError: bike_resting_hr is a continuous value that cannot be set to 0, but no value was found.

: 6
2019-08-15 14:55:55,798 - tensor_generators:114 - INFO - epochs: 1
2019-08-15 14:55:55,800 - tensor_generators:115 - INFO - Generator looped & shuffled over 7392 tensors.
2019-08-15 14:55:55,802 - tensor_generators:116 - INFO - True epoch number:1 in which 4999 tensors were presented.
5/5 [==============================] - 20s 4s/step - loss: 1.0

2019-08-15 14:57:14,349 - tensor_generators:114 - INFO - epochs: 3
2019-08-15 14:57:14,350 - tensor_generators:115 - INFO - Generator looped & shuffled over 7392 tensors.
2019-08-15 14:57:14,352 - tensor_generators:116 - INFO - True epoch number:3 in which 4999 tensors were presented.
5/5 [==============================] - 20s 4s/step - loss: 1.0414 - output_bike_max_pred_hr_continuous_loss: 0.4031 - output_bike_max_hr_continuous_loss: 0.2531 - output_bike_resting_hr_continuous_loss: 0.3851 - output_bike_max_pred_hr_continuous_logcosh: 0.4031 - output_bike_max_hr_continuous_logcosh: 0.2531 - output_bike_resting_hr_continuous_logcosh: 0.3851 - val_loss: 0.9480 - val_output_bike_max_pred_hr_continuous_loss: 0.3863 - val_output_bike_max_hr_continuous_loss: 0.2422 - val_output_bike_resting_hr_continuous_loss: 0.3195 - val_output_bike_max_pred_hr_continuous_logcosh: 0.3863 - val_output_bike_max_hr_continuous_logcosh: 0.2422 - val_output_bike_resting_hr_continuous_logcosh: 0.3195

Epoch 0000

2019-08-15 14:58:33,046 - tensor_generators:114 - INFO - epochs: 5
2019-08-15 14:58:33,048 - tensor_generators:115 - INFO - Generator looped & shuffled over 7392 tensors.
2019-08-15 14:58:33,049 - tensor_generators:116 - INFO - True epoch number:5 in which 4999 tensors were presented.
5/5 [==============================] - 20s 4s/step - loss: 0.9545 - output_bike_max_pred_hr_continuous_loss: 0.4100 - output_bike_max_hr_continuous_loss: 0.2285 - output_bike_resting_hr_continuous_loss: 0.3160 - output_bike_max_pred_hr_continuous_logcosh: 0.4100 - output_bike_max_hr_continuous_logcosh: 0.2285 - output_bike_resting_hr_continuous_logcosh: 0.3160 - val_loss: 0.9749 - val_output_bike_max_pred_hr_continuous_loss: 0.3851 - val_output_bike_max_hr_continuous_loss: 0.2414 - val_output_bike_resting_hr_continuous_loss: 0.3484 - val_output_bike_max_pred_hr_continuous_logcosh: 0.3851 - val_output_bike_max_hr_continuous_logcosh: 0.2414 - val_output_bike_resting_hr_continuous_logcosh: 0.3484

Epoch 0001

2019-08-15 14:59:52,139 - tensor_generators:114 - INFO - epochs: 7
2019-08-15 14:59:52,141 - tensor_generators:115 - INFO - Generator looped & shuffled over 7392 tensors.
2019-08-15 14:59:52,147 - tensor_generators:116 - INFO - True epoch number:7 in which 4999 tensors were presented.
5/5 [==============================] - 21s 4s/step - loss: 0.9772 - output_bike_max_pred_hr_continuous_loss: 0.3940 - output_bike_max_hr_continuous_loss: 0.2436 - output_bike_resting_hr_continuous_loss: 0.3395 - output_bike_max_pred_hr_continuous_logcosh: 0.3940 - output_bike_max_hr_continuous_logcosh: 0.2436 - output_bike_resting_hr_continuous_logcosh: 0.3395 - val_loss: 0.9685 - val_output_bike_max_pred_hr_continuous_loss: 0.3910 - val_output_bike_max_hr_continuous_loss: 0.2346 - val_output_bike_resting_hr_continuous_loss: 0.3429 - val_output_bike_max_pred_hr_continuous_logcosh: 0.3910 - val_output_bike_max_hr_continuous_logcosh: 0.2346 - val_output_bike_resting_hr_continuous_logcosh: 0.3429

Epoch 0001

5/5 [==============================] - 21s 4s/step - loss: 0.9365 - output_bike_max_pred_hr_continuous_loss: 0.3807 - output_bike_max_hr_continuous_loss: 0.2051 - output_bike_resting_hr_continuous_loss: 0.3507 - output_bike_max_pred_hr_continuous_logcosh: 0.3807 - output_bike_max_hr_continuous_logcosh: 0.2051 - output_bike_resting_hr_continuous_logcosh: 0.3507 - val_loss: 0.9663 - val_output_bike_max_pred_hr_continuous_loss: 0.3873 - val_output_bike_max_hr_continuous_loss: 0.2360 - val_output_bike_resting_hr_continuous_loss: 0.3430 - val_output_bike_max_pred_hr_continuous_logcosh: 0.3873 - val_output_bike_max_hr_continuous_logcosh: 0.2360 - val_output_bike_resting_hr_continuous_logcosh: 0.3430

Epoch 00018: val_loss did not improve from 0.93558
Epoch 19/20
4/5 [=======================>......] - ETA: 1s - loss: 0.9791 - output_bike_max_pred_hr_continuous_loss: 0.4422 - output_bike_max_hr_continuous_loss: 0.2189 - output_bike_resting_hr_continuous_loss: 0.3180 - output_bike_max_pred_hr_c

<Figure size 1296x1296 with 0 Axes>

In [ ]:
step = generate_test.__next__()
step[1].keys()
print(step[1]['output_bike_max_pred_hr_continuous'], step[1]['output_bike_resting_hr_continuous'])

In [ ]:
ecg_in = step[0]['input_full_ecg_bike_recovery']
hrr_out = step[1]['output_bike_max_hr_continuous']

In [ ]:
pred = model.predict(ecg_in)

In [ ]:
plt.scatter(hrr_out, pred[0])

In [47]:
recoveries = []
for i in range(10):
    step = generate_test.__next__()
    recoveries.append(step[1]['output_heart_rate_recovery_ecg_bike_autonomic_function'])